# Projeto 2 - Ciência dos Dados 

nome: Nicolas Byung Kwan Cho

O objetivo deste projeto é responder a pergunta: Com base nos dados de pluviosidade e temperatura na região de Campinas, como prever o número de casos de dengue?

Com o dataset abaixo é fácil ver que nossa variável target apresenta-se na coluna de 'casos confirmados', sendo as outras colunas as features.

In [17]:
#importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [18]:
#leitura do dataset
dados = pd.read_csv('dengue-dataset.csv')

In [19]:
dados.head()

,data,casos-confirmados,chuva,temperatura-media,temperatura-mininima,temperatura-maxima
0,1998-01-01,237,179.2,25.7,20.2,28.1
1,1998-02-01,331,226.3,25.1,20.0,28.0
2,1998-03-01,562,149.1,24.8,22.4,27.2
3,1998-04-01,187,46.4,22.7,18.1,26.0
4,1998-05-01,32,88.3,19.2,15.7,22.9


Das colunas acima, chuva é medida em mm e os dados de temperatura em graus celsius

Como em algumas datas não foram relatados os níveis de chuva, as linhas afetadas não participarão da análise, assim como a coluna da data que será considerada irrelevante para os casos confirmados.

In [24]:
dados.dropna(subset = ['chuva'], inplace=True)
dados.drop('data',inplace=True, axis=1)

In [25]:
dados.head()

,casos-confirmados,chuva,temperatura-media,temperatura-mininima,temperatura-maxima
0,237,179.2,25.7,20.2,28.1
1,331,226.3,25.1,20.0,28.0
2,562,149.1,24.8,22.4,27.2
3,187,46.4,22.7,18.1,26.0
4,32,88.3,19.2,15.7,22.9
